PyTorch Transformer from Scratch 

In [1]:
import torch
import torch.nn as nn

class SelfAttention(nn.Module):
  def __init__(self, embed_size, heads):
    # initialize parent class
    super(SelfAttention, self).__init__()
    self.embed_size = embed_size
    self.heads = heads
    self.head_dim = embed_size // heads

    # check that 256 embedding size can't be split into 7 parts, or any other number that is not possible
    assert (self.head_dim * heads == embed_size), "Incorrect embedding size. Needs to be divisible by heads"
    
    # linear layer that we send out VKQ through
    # defining VKQ below
    self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
    self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
    self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
    self.fc_out = nn.Linear(heads*self.head_dim, embed_size)

  def forward(self, values, keys, query, mask):
    # define number of training examples
    N = query.shape[0]
    value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

    # split embedding into self.heads pieces
    values = values.reshape(N, value_len, self.heads, self.head_dim) 
    keys = keys.reshape(N, key_len, self.heads, self.head_dim)
    queries = query.reshape(N, key_len, self.heads, self.head_dim)

    # matrix multiplication of q and V
    # einstein summation notation
    energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

    if mask is not None:
      energy = energy.masked_fill(mask == 0, float("-1e20"))

    # running this through the softmax() in the self attention function
    attention = torch.softmax(energy / (self.embed_size**(1/2)), dim=3)

    # output: N, query_len, heads, head_dim) 
    out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(N, query_len, self.heads*self.head_dim)

    out = self.fc_out(out)
    return out

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
